## Llama7B on the squad dataset

The purpose of this is to measure the coherence of an LLM on the Cohence of solving general problems

# Model set up

In [1]:

# import the hugging face transformers library
import wandb
import torch
import os
from transformers import  Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader



In [2]:
#making sure I am using the gpu
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA A40


In [3]:
#Including code to get from token from environment
token= ""

In [4]:


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",token=token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",token=token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

## Gather dataset

In [6]:
squad_dataset = load_dataset("squad")

### Visualizing the Squad dataset

In [7]:
squad_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [8]:
squad_dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [9]:
def preprocess_function(examples):
    questions = examples['question']
    contexts = examples['context']

    # Tokenize the questions and contexts
    inputs = tokenizer(
        questions,
        contexts,
        padding="max_length",
        truncation=True,
        max_length=30,
        return_tensors="pt"
    )

    # Extract the answer text and start positions
    answers = examples['answers']
    start_positions = []
    labels = []

    for i in range(len(answers)):
        answer = answers[i]
        start_position = answer['answer_start'][0]
        labels.append(answer['text'][0])
        start_positions.append(start_position)

    inputs['start_positions'] = start_positions
    inputs['labels'] = labels

    return inputs


squad_dataset_validation = squad_dataset['validation']
squad_dataset_train = squad_dataset['train']

# Map preprocessing function to the dataset
encoded_train_dataset = squad_dataset_train.map(preprocess_function, batched=True, remove_columns=squad_dataset_train.column_names)
encoded_validation_dataset = squad_dataset_validation.map(preprocess_function, batched=True, remove_columns=squad_dataset_validation.column_names)

In [10]:
encoded_validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'labels'],
    num_rows: 10570
})

In [11]:
encoded_validation_dataset[0]

{'input_ids': [1,
  8449,
  25167,
  3815,
  9875,
  278,
  319,
  8610,
  472,
  5670,
  27207,
  29871,
  29945,
  29900,
  29973,
  1,
  5670,
  27207,
  29871,
  29945,
  29900,
  471,
  385,
  3082,
  5733,
  3748,
  304,
  8161,
  278,
  8064],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'start_positions': 177,
 'labels': 'Denver Broncos'}

### Measuring Existing performance of LLama7B

In [12]:
def calculate_metrics(predicted_answer, true_answer):
    predicted_tokens = predicted_answer.lower().split()
    true_tokens = true_answer.lower().split()

    common_tokens = set(predicted_tokens) & set(true_tokens)
    exact_match = int(predicted_tokens == true_tokens)

    if len(predicted_tokens) == 0 or len(true_tokens) == 0:
        f1_score = int(predicted_tokens == true_tokens)
    else:
        precision = len(common_tokens) / len(predicted_tokens)
        recall = len(common_tokens) / len(true_tokens)
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return exact_match, f1_score

In [13]:
#this code below is meant to just check the existing perfomance of llama 7b on the dataset

def evaluate_model(model, tokenizer, dataset, num_samples=100):
    exact_match_scores = []
    f1_scores = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    current = 0
    for example in dataset["validation"].select(range(num_samples)):
        #print("p1")
        question = example["question"]
        context = example["context"]
        true_answer = example["answers"]["text"][0]

        #print("p2")
        input_text = f"Question: {question}\nContext: {context}\nAnswer:"
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

        #print("p3")
        with torch.no_grad():
            output_ids = model.generate(input_ids, max_new_tokens=50, num_return_sequences=1)

        predicted_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        #print("p4")
        exact_match, f1_score = calculate_metrics(predicted_answer, true_answer)
        exact_match_scores.append(exact_match)
        f1_scores.append(f1_score)
        current+=1
        print("Current value,",current)

    print("p5")
    avg_exact_match = sum(exact_match_scores) / len(exact_match_scores)
    avg_f1_score = sum(f1_scores) / len(f1_scores)
    

    return avg_exact_match, avg_f1_score

num_samples = 10  # Specify the number of samples to evaluate
exact_match, f1_score = evaluate_model(model, tokenizer, squad_dataset, num_samples)
print(f"Exact Match: {exact_match:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Current value, 1
Current value, 2
Current value, 3
Current value, 4
Current value, 5
Current value, 6
Current value, 7
Current value, 8
Current value, 9
Current value, 10
p5
Exact Match: 0.0000
F1 Score: 0.0287


### Sparse Reward Function

In [14]:
def sparse_reward(predictions, references, threshold=0.8):
    f1_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        common_tokens = set(pred_tokens) & set(ref_tokens)
        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(ref_tokens)
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(1 if f1_score >= threshold else 0)
    return torch.tensor(f1_scores)

## Dense Reward Function

In [15]:
#testing new version of the dense reward
# Define the dense reward function
def dense_reward(predictions, references):
    f1_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        common_tokens = set(pred_tokens) & set(ref_tokens)
        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(ref_tokens)
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1_score)
    return torch.tensor(f1_scores)

In [16]:
stop

NameError: name 'stop' is not defined

## Model Fine-tuning

In [ ]:
from transformers import Trainer, TrainingArguments
import wandb

def train_model(model, tokenizer, input_train_dataset,input_validation_dataset, reward_function, reward_type):
    # Initialize WandB with specific configurations
    wandb.init(project="Coherence", name=f"Training with {reward_type} Reward")
    
    # Define the training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_{reward_type}",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="steps",
        logging_steps=500,
        save_steps=1000,
        save_total_limit=2,
        seed=42,
        load_best_model_at_end=True,
        report_to="wandb"
    )
    
    # Create the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=input_train_dataset,
        eval_dataset=input_validation_dataset,
        tokenizer=tokenizer,
        compute_metrics=lambda pred: {"reward": reward_function(pred.predictions, pred.label_ids)}
    )
    
    # Train the model
    trainer.train()
    
    # Finish the WandB run
    wandb.finish()

# Example usage
train_model(model, tokenizer, encoded_train_dataset, encoded_validation_dataset, dense_reward, "Dense")

#now train the sparse reward
#squad_dataset_validation = squad_dataset['validation']
#squad_dataset_train = squad_dataset['train']


## Creating mock versions of both for error checking

In [ ]:
# Extract a single sample from training and validation datasets
train_sample = encoded_train_dataset[0]
validation_sample = encoded_validation_dataset[0]

# Print out the samples to understand their structure
print("Training Sample:", train_sample)
print("Validation Sample:", validation_sample)


In [ ]:
# Simulate a prediction output from the model, for demonstration let's just reuse the input_ids as predictions
simulated_predictions = [train_sample['input_ids'], validation_sample['input_ids']]
references = [train_sample['labels'], validation_sample['labels']]


In [ ]:
def process_predictions_and_references(predictions, references, tokenizer):
    # Decode predictions (list of token IDs) to text
    decoded_predictions = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
    return decoded_predictions, references  # References are already text

# Example usage (assuming some model outputs and references)
predictions = [train_sample['input_ids'], validation_sample['input_ids']]  # Simulated model predictions
references = [train_sample['labels'], validation_sample['labels']]  # Actual references

decoded_predictions, processed_references = process_predictions_and_references(predictions, references, tokenizer)

# Now pass the decoded predictions and the text references to the reward function
rewards = dense_reward(decoded_predictions, processed_references)
print("Computed Rewards:", rewards)


#### Testing  not just function but on a batch of data

In [ ]:
# If using a DataLoader, you can set batch_size directly.
# For datasets library, manually slice the dataset for simplicity
batch_size = 10  # Example batch size
train_batch = encoded_train_dataset[:batch_size]
validation_batch = encoded_validation_dataset[:batch_size]

# Extract predictions (simulated here) and references
predictions = [item['input_ids'] for item in train_batch]  # Assuming a simple model prediction
references = [item['labels'] for item in train_batch]  # Actual labels

In [ ]:
def decode_inputs(input_ids, tokenizer):
    """ Decode a batch of input IDs to text. """
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]

# Decoding the input IDs for the whole batch
decoded_texts = decode_inputs(train_batch['input_ids'], tokenizer)

# You already have the labels in the correct format
labels = train_batch['labels']

In [ ]:
# Calculate rewards
rewards = dense_reward(decoded_texts, labels)
print("Batch Computed Rewards:", rewards)


In [ ]:
def process_batch(batch, tokenizer):
    """ Process a single batch of data, returning decoded texts and labels. """
    decoded_texts = decode_inputs(batch['input_ids'], tokenizer)
    labels = batch['labels']
    return decoded_texts, labels

def evaluate_batch(batch, tokenizer):
    """ Evaluate a single batch using the dense_reward function. """
    decoded_texts, labels = process_batch(batch, tokenizer)
    rewards = dense_reward(decoded_texts, labels)
    return rewards

# Example usage, assuming you're iterating over batches of your dataset
for i in range(0, len(encoded_train_dataset), batch_size):
    batch = encoded_train_dataset[i:i+batch_size]
    batch_rewards = evaluate_batch(batch, tokenizer)
    print(f"Batch {i // batch_size} Computed Rewards:", batch_rewards)


In [ ]:
#top

In [17]:
# Extract a single sample from training and validation datasets
train_sample = encoded_train_dataset[0]
validation_sample = encoded_validation_dataset[0]

# Print out the samples to understand their structure
print("Training Sample:", train_sample)
print("Validation Sample:", validation_sample)

Training Sample: {'input_ids': [1, 1763, 6029, 1258, 278, 9167, 6182, 16831, 23244, 2615, 297, 29871, 29896, 29947, 29945, 1, 2595, 4496, 332, 635, 29892, 278, 3762, 756, 263, 11865, 2931, 29889, 2180, 459], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'start_positions': 515, 'labels': 'Saint Bernadette Soubirous'}
Validation Sample: {'input_ids': [1, 8449, 25167, 3815, 9875, 278, 319, 8610, 472, 5670, 27207, 29871, 29945, 29900, 29973, 1, 5670, 27207, 29871, 29945, 29900, 471, 385, 3082, 5733, 3748, 304, 8161, 278, 8064], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'start_positions': 177, 'labels': 'Denver Broncos'}


In [18]:
def process_predictions_and_references(predictions, references, tokenizer):
    # Decode predictions (list of token IDs) to text
    decoded_predictions = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
    return decoded_predictions, references  # References are already text

# Example usage (assuming some model outputs and references)
predictions = [train_sample['input_ids'], validation_sample['input_ids']]  # Simulated model predictions
references = [train_sample['labels'], validation_sample['labels']]  # Actual references

decoded_predictions, processed_references = process_predictions_and_references(predictions, references, tokenizer)

# Now pass the decoded predictions and the text references to the reward function
rewards = dense_reward(decoded_predictions, processed_references)
print("Computed Rewards:", rewards)

Computed Rewards: tensor([0, 0])


In [19]:
# If using a DataLoader, you can set batch_size directly.
# For datasets library, manually slice the dataset for simplicity
batch_size = 10  # Example batch size
train_batch = encoded_train_dataset[:batch_size]
validation_batch = encoded_validation_dataset[:batch_size]

# Extract predictions (simulated here) and references
#predictions = [item['input_ids'] for item in train_batch]  # Assuming a simple model prediction
#references = [item['labels'] for item in train_batch]  # Actual labels

In [20]:
# Assuming train_batch is a dictionary with keys 'input_ids' and 'labels' each mapping to a list
predictions = train_batch['input_ids']  # Directly access the list of input_ids
references = train_batch['labels']      # Directly access the list of labels


In [ ]:
# Example of handling batches correctly
#batch_size = 10
for i in range(0, len(encoded_train_dataset), batch_size):
    # Ensure that the slicing returns the correct format
    batch = encoded_train_dataset[i:i+batch_size]
    if isinstance(batch, dict):
        predictions = batch['input_ids']
        references = batch['labels']
    else:
        predictions = [item['input_ids'] for item in batch]
        references = [item['labels'] for item in batch]

    # Continue processing as before
    decoded_predictions, processed_references = process_predictions_and_references(predictions, references, tokenizer)
    batch_rewards = dense_reward(decoded_predictions, processed_references)
    print(f"Batch {i//batch_size} Computed Rewards:", batch_rewards)


In [ ]:
Batch 0 Computed Rewards: tensor([0.0000, 0.1600, 0.0952, 0.0800, 0.2308, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])
Batch 1 Computed Rewards: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0833, 0.0000, 0.0000, 0.2222, 0.0000,
        0.0000])
Batch 2 Computed Rewards: tensor([0.0000, 0.0000, 0.0833, 0.0000, 0.0000, 0.1600, 0.0714, 0.0000, 0.0690,
        0.5161])
Batch 3 Computed Rewards: tensor([0.2759, 0.0000, 0.0833, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])
Batch 4 Computed Rewards: tensor([0.0000, 0.1290, 0.0000, 0.0741, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.3333])
Batch 5 Computed Rewards: tensor([0.0000, 0.0000, 0.0000, 0.0909, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.2609])
Batch 6 Computed Rewards: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.3810, 0.0000, 0.0000, 0.2609, 0.0000,
        0.0000])
Batch 7 Computed Rewards: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0741, 0.0000, 0.1176,
        0.0000])

In [ ]:
#index

In [ ]:
#changing the reward function to handle tokens instead of text.
def dense_reward(predictions, references, tokenizer):
    f1_scores = []
    for pred, ref in zip(predictions, references):
        # Decode the tokens to strings
        pred_text = tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        ref_text = tokenizer.decode(ref, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        pred_tokens = pred_text.split()
        ref_tokens = ref_text.split()
        common_tokens = set(pred_tokens) & set(ref_tokens)
        precision = len(common_tokens) / len(pred_tokens) if len(pred_tokens) > 0 else 0
        recall = len(common_tokens) / len(ref_tokens) if len(ref_tokens) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1_score)
    return torch.tensor(f1_scores)


In [ ]:
#starting from the top
#we have the dense function, so lets work backwards, and delete all else
def process_batch(batch, tokenizer):
    """ Process a single batch of data, returning decoded texts and labels. """
    decoded_texts = decode_inputs(batch['input_ids'], tokenizer)
    labels = batch['labels']
    return decoded_texts, labels

def evaluate_batch(batch, tokenizer):
    """ Evaluate a single batch using the dense_reward function. """
    decoded_texts, labels = process_batch(batch, tokenizer)
    rewards = dense_reward(decoded_texts, labels)
    return rewards

# Example usage, assuming you're iterating over batches of your dataset
for i in range(0, len(encoded_train_dataset), batch_size):
    batch = encoded_train_dataset[i:i+batch_size]
    batch_rewards = evaluate_batch(batch, tokenizer)
    print(f"Batch {i // batch_size} Computed Rewards:", batch_rewards)


In [ ]:
#The End